In [1]:
import string
import re
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.tokenize import word_tokenize
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report,accuracy_score
import string 

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\16514\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\16514\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Read Dataset

In [2]:
imdb_data = pd.read_csv('./data/IMDB Dataset.csv')
imdb_data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
imdb_data['sentiment'].value_counts()

negative    25000
positive    25000
Name: sentiment, dtype: int64

### Split the data

In [4]:
train_reviews=imdb_data.review[:30000]
train_sentiments=imdb_data.sentiment[:30000]

val_reviews=imdb_data.review[30000:40000]
val_sentiments=imdb_data.sentiment[30000:40000]

test_reviews=imdb_data.review[40000:]
test_sentiments=imdb_data.sentiment[40000:]

## Preprocess Dataset

In [5]:
def preprocess(reviews):
    container = []
    for review in reviews:
        review = review.replace("<br />", "")
        for ele in string.punctuation:
                if ele in review:
                        review = review.replace(ele, "")
        container.append(review)
    return container

In [6]:
train_reviews = preprocess(train_reviews)
val_reviews = preprocess(val_reviews)
test_reviews = preprocess(test_reviews)

In [7]:
'''
Method that take in the training dataset, then return the positive and negative words log probability.
Input: train_reviews: reviews (sentences) for training
       train_sentiments: sentiments (label) for training
       val_reviews: reviews (sentences) for validation
       test_reviews: reviews (sentences) for testing
       tfidf: boolean variable indicating whether using bow or tfidf
       alpha: laplance smoothing variable, default to be 1.0
       ngram_range: the scale of ngram model will be used, default = (1,1) unigram
return: negative_word_log_prob_dict: dictionary that contains the word:log probability pair for negative class
        positive_word_log_prob_dict: dictionary that contains the word:log probability pair for positive class
        mnb: the trained multinomial naive bayes model, later can be used for testing
        transformed_val_reviews: transformed val reviews that later can be used for validation
        transformed_test_reviews: transformed test reviews that later can be used for testing
        vec: either the tfidfVectorize build from tfidf model or the CountVectorizer build from Bag of word model.
'''

def generate_log_prob(train_reviews, train_sentiments, val_reviews, test_reviews, tfidf=False, alpha=1.0, ngram_range = (1,1)):

    if (tfidf):
        #Tfidf vectorizer
        vec=TfidfVectorizer(use_idf=tfidf, ngram_range=ngram_range)
        transformed_train_reviews=vec.fit_transform(train_reviews)
        transformed_val_reviews = vec.transform(val_reviews)
        transformed_test_reviews=vec.transform(test_reviews)
    else:
        vec=CountVectorizer(ngram_range=(1,1))
        transformed_train_reviews=vec.fit_transform(train_reviews)
        transformed_val_reviews = vec.transform(val_reviews)
        transformed_test_reviews=vec.transform(test_reviews)

    #training the model
    mnb = MultinomialNB(alpha=alpha)

    #fitting the naive bayes for bag of words
    mnb = mnb.fit(transformed_train_reviews, train_sentiments)
    negative_log_prob = mnb.feature_log_prob_[0]
    positive_log_prob = mnb.feature_log_prob_[1]

    # Generate two dict: word:log_prob
    negative_word_log_prob_dict = {}
    positive_word_log_prob_dict = {}
    for word, index in vec.vocabulary_.items():
        negative_word_log_prob_dict[word] = negative_log_prob[index]
        positive_word_log_prob_dict[word] = positive_log_prob[index]
    
    return negative_word_log_prob_dict, positive_word_log_prob_dict, mnb, transformed_val_reviews, transformed_test_reviews, vec

In [8]:
def calculate_reweight_dict(positive_prob_dict, negative_prob_dict, dataset, threshold=0.1):
    wrongly_classified_token_dict = {} # counts how many times a token has negative impact on wrongly classified sentence
    token_dict = {} # counts how many times a token has appeared in total
    reweight_dict = {}
    for sentence in dataset:
        real_label = sentence[6:14]
        tokens = sentence[17:].split(" ")
        for token in tokens:
            if token not in token_dict:
                token_dict[token] = 1
            else:
                token_dict[token] += 1
        if real_label == "positive": # marked as negative
            for token in tokens:
                if token not in positive_prob_dict: continue
                elif positive_prob_dict[token] < negative_prob_dict[token]:
                    if token not in wrongly_classified_token_dict:
                        wrongly_classified_token_dict[token] = 1
                    else:
                        wrongly_classified_token_dict[token] += 1
        elif real_label == "negative": # marked as positive
            for token in tokens:
                if token not in positive_prob_dict: continue
                elif negative_prob_dict[token] < positive_prob_dict[token]:
                    if token not in wrongly_classified_token_dict:
                        wrongly_classified_token_dict[token] = 1
                    else:
                        wrongly_classified_token_dict[token] += 1
    # print(wrongly_classified_token_dict)
    for word, prob in wrongly_classified_token_dict.items():
        # print(word, prob)
        weight = 0
        if positive_prob_dict[word] < negative_prob_dict[word]:
            weight = positive_prob_dict[word] - negative_prob_dict[word]
        elif negative_prob_dict[word] < positive_prob_dict[word]:
            weight = negative_prob_dict[word] - positive_prob_dict[word]
        multiplier = 1 - wrongly_classified_token_dict[word] / token_dict[word]
        if multiplier == 0:
            reweight_dict[word] = 0
        else:
            reweight_dict[word] = np.exp(weight * multiplier)
    return reweight_dict
    
    

In [9]:
'''
This method will take in a word:scale dict, then take in the negative and positive word:log_probability dict, manually change the weight of the words in the model and the dict
Input: word_change_scale: this is the word-scale dictionary, how much the weight of the word should be changed, For example if the value is 0.5, we will say 
                          the probability of the word in negative class should multiply 0.5, in original probability, we take power to the scale
       model: the trained naive bayes model, which the feature_log_prob_ attribute will be manually changed based on previous two params
       negative_word_log_prob_dict: dictionary that contains the word:log probability pair for negative class, which some values will be changed
       positive_word_log_prob_dict: dictionary that contains the word:log probability pair for positive class, which some values will be changed
       vec: either the tfidfVectorize build from tfidf model or the CountVectorizer build from Bag of word model.
return: negative_word_change_scale: The modified negative dict
        positive_word_change_scale: The modified positive dict
        model: The modified naive bayes model
'''

def change_weight(word_change_scale, model, negative_word_log_prob_dict, positive_word_log_prob_dict, vec):
    for word, scale in word_change_scale.items():
        # change the weight of words in negative and positive word:log_prob dict
        negative_word_log_prob_dict[word] *= scale
        positive_word_log_prob_dict[word] *= scale

        # change the weight of words in the model
        index_in_model = vec.vocabulary_[word]
        model.feature_log_prob_[0][index_in_model] *= scale
        model.feature_log_prob_[1][index_in_model] *= scale

    return negative_word_log_prob_dict, positive_word_log_prob_dict, model

### Bag of Word - Naive Bayes

In [10]:
neg, pos, mnb, transformed_val_reviews, transformed_test_reviews, vec= generate_log_prob(train_reviews, train_sentiments, val_reviews, test_reviews, alpha = 0.05)

In [11]:
val_pred = mnb.predict(transformed_val_reviews)
test_pred = mnb.predict(transformed_test_reviews)
val_acc = accuracy_score(val_sentiments, val_pred)
test_acc = accuracy_score(test_sentiments, test_pred)
print(f"val accuracy is {val_acc}, test accuracy is {test_acc}")


val accuracy is 0.8411, test accuracy is 0.8419


In [12]:
#Classification report for bag of words 
mnb_bow_report=classification_report(val_sentiments,val_pred,target_names=['Positive','Negative'])
print(mnb_bow_report)

              precision    recall  f1-score   support

    Positive       0.83      0.86      0.85      5022
    Negative       0.86      0.82      0.84      4978

    accuracy                           0.84     10000
   macro avg       0.84      0.84      0.84     10000
weighted avg       0.84      0.84      0.84     10000



## Reweight the model using perceptron

In [13]:
def naive_inference(reweight, pos, neg, test_reviews, test_sentiments, test_start=40000):
    correct = 0
    for i in range(len(test_reviews)):
        word_list = test_reviews[i].strip().split()
        final_result = 0
        
        for word in word_list:
            weight = 1
            pprob = 0
            nprob = 0
            if word in reweight:
                weight = reweight[word]
            if word in pos:
                pprob = pos[word]
            if word in neg:
                nprob = neg[word]
            final_result += weight*pprob - weight*nprob
        if (final_result > 0 and test_sentiments[i+test_start] == "positive") or (final_result < 0 and test_sentiments[i+test_start] == "negative"):
            correct += 1
    print(correct/len(test_reviews))
    return correct/len(test_reviews)

In [14]:
def perceptron_update(positive_prob_dict, negative_prob_dict, val_reviews, val_sentiments, test_reviews, test_sentiments, max_iter=10, learning_rate=0.01, val_start=30000):
    """
    need to handle unseen word here
    has to be a method
    """
    initial_weight = {}
    for word in positive_prob_dict.keys():
        initial_weight[word] = 1
    for word in negative_prob_dict.keys():
        initial_weight[word] = 1
    for i in range(max_iter):
        random_array = np.arange(len(val_sentiments))
        np.random.shuffle(random_array)
        correct = 0
        for index in random_array:
            word_list = val_reviews[index].strip().split()
            final_result = 0
            for word in word_list:
                weight = 1
                pprob = 0
                nprob = 0
                if word in initial_weight:
                    weight = initial_weight[word]
                if word in positive_prob_dict:
                    pprob = positive_prob_dict[word]
                if word in negative_prob_dict:
                    nprob = negative_prob_dict[word]
                final_result += weight*pprob - weight*nprob
            if (final_result > 0 and val_sentiments[index+val_start] == "positive") or (final_result < 0 and val_sentiments[index+val_start] == "negative"):
                correct += 1
                for word in word_list:
                    if word in positive_prob_dict and word in negative_prob_dict:
                        initial_weight[word] -= learning_rate/len(val_sentiments)*initial_weight[word]
                continue
            else:
                for word in word_list:
                    if word in positive_prob_dict and word in negative_prob_dict:
                        if val_sentiments[index+val_start] == "positive":
                            initial_weight[word] += learning_rate*((positive_prob_dict[word]-negative_prob_dict[word]) - 1/len(val_sentiments)*initial_weight[word])
                        else:
                            initial_weight[word] += learning_rate*((negative_prob_dict[word]-positive_prob_dict[word]) - 1/len(val_sentiments)*initial_weight[word])
        test_acc = naive_inference(initial_weight, positive_prob_dict, negative_prob_dict, test_reviews, test_sentiments)
        print(f"finish training epoch {i}, the val accuracy is {correct/len(val_sentiments)}, the test accuracy is {test_acc}")
        
    return initial_weight

In [15]:
reweight = perceptron_update(pos, neg, val_reviews, val_sentiments, test_reviews, test_sentiments)

0.8542
finish training epoch 0, the val accuracy is 0.846, the test accuracy is 0.8542
0.8573
finish training epoch 1, the val accuracy is 0.859, the test accuracy is 0.8573


In [ ]:
neg, pos, mnb = change_weight(reweight, mnb, neg, pos, vec)


In [ ]:
val_pred = mnb.predict(transformed_val_reviews)
test_pred = mnb.predict(transformed_test_reviews)
val_acc = accuracy_score(val_sentiments, val_pred)
test_acc = accuracy_score(test_sentiments, test_pred)
print(f"val accuracy is {val_acc}, test accuracy is {test_acc}")